In [ ]:
import csv
import itertools
import json
import os
import pathlib
import sys

from matplotlib import pyplot as plt
import numpy as np
import potpourri3d as pp3d

sys.path.append(str(pathlib.PurePath('..', '..', 'src')))
from linear_geodesic_optimization.data import utility
from linear_geodesic_optimization.graph import boundary
from linear_geodesic_optimization.mesh.rectangle import Mesh as RectangleMesh

In [ ]:
# Outputs are stored in `directory_outputs` / <output number> / `subdirectory_output`
epsilon = 7
directory_data = pathlib.PurePath('..')
directory_outputs = pathlib.PurePath('..', '..', 'outputs', 'esnet_windowed')
subdirectory_output = pathlib.PurePath(f'{epsilon}_0.0002_50_50')
directories_outputs = list(sorted([
    (
        float(directory_output),
        directory_outputs / directory_output / subdirectory_output,
    )
    for i, directory_output in enumerate(sorted(os.listdir(directory_outputs)))
    if os.path.isdir(directory_outputs / directory_output)
]))

directory_links_standin = pathlib.PurePath('esnet', 'links')

directory_output_images = pathlib.PurePath('..', '..', 'outputs', 'images', 'postprocessed_intercept')
directory_output_csvs = pathlib.PurePath('..', '..', 'outputs', 'csvs')

height_scale = 0.20

In [ ]:
def get_nearest_vertex(mesh: RectangleMesh, vertex):
    """
    Find the xy-coordinates of the nearest mesh point to some coordinates.
    """
    nearest_vertex = mesh.get_coordinates()[mesh.nearest_vertex(vertex).index]
    return [nearest_vertex[0], nearest_vertex[1]]

def compute_geodesics_from_graph(mesh: RectangleMesh, network_vertices, network_edges, geodesic_index_pairs):
    mesh_scale = mesh.get_scale()

    network_indices = set()
    bad_indices = set()
    for (index_source, index_target) in geodesic_index_pairs:
        network_indices.add(index_source)
        network_indices.add(index_target)
    for network_index in network_indices:
        network_vertex = network_vertices[network_index]
        if network_vertex.tolist() not in (mesh.get_coordinates()[:, :2].tolist()):
            try:
                mesh.add_vertex_at_coordinates(network_vertex)
            except ValueError:
                bad_indices.add(network_index)

    path_solver = pp3d.EdgeFlipGeodesicSolver(
        mesh.get_coordinates(),
        np.array([
            [vertex.index for vertex in face.vertices()]
            for face in mesh.get_topology().faces()
        ])
    )

    geodesics = []
    for (index_source, index_target) in geodesic_index_pairs:
        if index_source in bad_indices or index_target in bad_indices:
            continue

        source = mesh.nearest_vertex(network_vertices[index_source]).index
        target = mesh.nearest_vertex(network_vertices[index_target]).index

        if source == target:
            continue
        else:
            geodesic = path_solver.find_geodesic_path(source, target)
            geodesics.append((geodesic[:, :2] / mesh_scale).tolist())

    return geodesics

In [ ]:
# Grab the data from the files
outputs = []
times = []
for t, directory_output in directories_outputs:
    with open(directory_output / 'output.json', 'r') as file_output:
        outputs.append(json.load(file_output))
        times.append(t)

In [ ]:
# Clustering should always return the same set of clusters, but we might
# end up omiting nodes that have no associated measurements at a
# snapshot. We might also choose a different cluster representative.
# So, we need to ensure our network vertices are uniform across time.

# Determine a consistent set of cluster representatives, their
# coordinates, and their associated data
node_labels_to_representatives = {}
node_representatives_to_labels = {}
node_labels_to_coordinates = {}
node_labels_to_data = {}
node_data_keys = set()
for output in outputs:
    network = output['network']
    graph_data, vertex_data, edge_data = network

    # Cases depending on whether clustering was used
    if 'elements' in vertex_data:
        for index, (node_label, elements, coordinates) in enumerate(zip(graph_data['labels'], vertex_data['elements'], graph_data['coordinates'])):
            node_representative = min(elements)
            for element in elements:
                node_labels_to_representatives[element] = node_representative
            node_labels_to_coordinates[node_representative] = coordinates
            node_labels_to_data[node_representative] = {
                key: value[index]
                for key, value in vertex_data.items()
            }
    else:
        for index, (node_label, coordinates) in enumerate(zip(graph_data['labels'], graph_data['coordinates'])):
            node_labels_to_representatives[node_label] = node_label
            node_labels_to_coordinates[node_label] = coordinates
            node_labels_to_data[node_label] = {
                key: value[index]
                for key, value in vertex_data.items()
            }

    for key in vertex_data.keys():
        node_data_keys.add(key)

node_indices_to_labels = list(sorted(node_labels_to_coordinates))
node_labels_to_indices = {label: index for index, label in enumerate(node_indices_to_labels)}
for node_label, node_representative in node_labels_to_representatives.items():
    node_labels_to_indices[node_label] = node_labels_to_indices[node_representative]
for node_label, node_representative in node_labels_to_representatives.items():
    if node_representative not in node_representatives_to_labels:
        node_representatives_to_labels[node_representative] = [node_label]
    else:
        node_representatives_to_labels[node_representative].append(node_label)

# Combine the computed data into an appropriate format
node_labels = node_indices_to_labels
node_coordinates = [node_labels_to_coordinates[label] for label in node_labels]
node_data = {
    key: [
        node_labels_to_data[label][key] if key in node_labels_to_data[label] else None
        for label in node_labels
    ]
    for key in node_data_keys
}

# Relabel the networks
for output in outputs:
    network = output['network']
    graph_data, vertex_data, edge_data = network

    graph_data_new = {
        'coordinates': node_coordinates,
        'edges': [
            (node_labels_to_indices[source_label], node_labels_to_indices[target_label])
            for source_index, target_index in graph_data['edges']
            for source_label in (graph_data['labels'][source_index],)
            for target_label in (graph_data['labels'][target_index],)
        ],
        'labels': node_indices_to_labels,
        'bounding_box': graph_data['bounding_box'] if 'bounding_box' in graph_data else None
    }
    vertex_data_new = node_data
    edge_data_new = edge_data

    output['network'] = (graph_data_new, vertex_data_new, edge_data_new)

In [ ]:
# Normalize the z-coordinates across time
zs = []
for output in outputs:
    z = np.array(output['final']) - np.array(output['initial'])
    zs.append(z - np.mean(z))

In [ ]:
# Assume most parameters don't change across snapshots
parameters = outputs[0]['parameters']
width = parameters['width']
height = parameters['height']
mesh_scale = parameters['mesh_scale']
coordinates_scale = parameters['coordinates_scale']
mesh = RectangleMesh(width, height, mesh_scale)

# Assume bounding box information is constant
graph_data, vertex_data, edge_data = outputs[0]['network']
bounding_box = graph_data['bounding_box']
network_vertices = mesh.map_coordinates_to_support(np.array(node_coordinates), coordinates_scale, bounding_box)

In [ ]:
# Compute the network borders
borders = []
distances_to_borders = []
for output in outputs:
    network = output['network']
    graph_data, vertex_data, edge_data = network

    network_edges = graph_data['edges']
    network_border = boundary.compute_border(network_vertices, network_edges)
    distances_to_borders.append(np.array([
        boundary.distance_to_border(
            np.array(vertex_coordinate),
            network_border
        )
        for vertex_coordinate in mesh.get_coordinates()[:, :2]
    ]))
    borders.append(np.where(distances_to_borders[-1] == 0.)[0])

In [ ]:
# Determine values for vertical scaling
z_max = -np.inf
z_min = np.inf
for z, hull in zip(zs, borders):
    z_max = max(z_max, np.max(z[hull]))
    z_min = min(z_min, np.min(z[hull]))

In [ ]:
links_data = []
for t, output, z, distance_to_network in zip(times, outputs, zs, distances_to_borders):
    z_original = np.copy(z)
    parameters = output['parameters']
    filename_links = pathlib.PurePath(parameters['filename_links'])

    if directory_links_standin is not None:
        filename_links = directory_links_standin / filename_links.name

    # Postprocess
    hull = [index for index in range(mesh.get_topology().n_vertices) if distance_to_network[index] < 0.025]
    distance_to_network = np.maximum(distance_to_network - 0.025, 0.)
    z = z - z_min
    z = z / (z_max - z_min) * height_scale
    z = (z + 0.05) * np.exp(-1000 * distance_to_network**2) - 0.05

    mesh.set_parameters(z)
    mesh.trim_to_set(hull)

    path_solver = pp3d.EdgeFlipGeodesicSolver(
        mesh.get_coordinates(),
        np.array([
            [vertex.index for vertex in face.vertices()]
            for face in mesh.get_topology().faces()
        ])
    )

    with open(directory_data / filename_links, 'r') as f:
        reader = csv.DictReader(f)
        geodesics_data = {}
        for row in reader:
            key = (row['source_id'], row['target_id'])

            network_edge = (
                node_labels_to_indices[key[0]],
                node_labels_to_indices[key[1]]
            )
            vertex_source = mesh.nearest_vertex(network_vertices[network_edge[0]])
            vertex_target = mesh.nearest_vertex(network_vertices[network_edge[1]])
            geodesic_path = path_solver.find_geodesic_path(vertex_source.index, vertex_target.index) if vertex_source.index != vertex_target.index else [mesh.get_coordinates()[vertex_source.index]]
            geodesic_distance = sum([np.linalg.norm(u - v) for u, v in itertools.pairwise(geodesic_path)], 0.)

            row['geodesic_distance'] = geodesic_distance
            geodesics_data[key] = row
    links_data.append(geodesics_data)

    mesh.restore_removed_vertices()

In [ ]:
x = np.array([row['geodesic_distance'] for geodesics_data in links_data for row in geodesics_data.values() if row['rtt']])
y = np.array([float(row['rtt']) for geodesics_data in links_data for row in geodesics_data.values() if row['rtt']])
plt.plot(x, y, 'b.')
plt.show()

In [ ]:
m, c = np.linalg.lstsq(
    np.vstack([x, np.ones(len(x))]).T,
    y
)[0]
# m = np.linalg.lstsq(x.reshape((-1, 1)), y)[0]
# c = 0

In [ ]:
keys = set(
    key
    for geodesics_data in links_data
    for key in geodesics_data
)

In [ ]:
# Write time series plots
# for key in keys:
#     geodesics_key = np.array([geodesics_data[key]['geodesic_distance'] for geodesics_data in links_data])
#     rtts_key = np.array([float(geodesics_data[key]['rtt']) if geodesics_data[key]['rtt'] else 0. for geodesics_data in links_data])

#     fig, ax = plt.subplots(1, 1)
#     ax.plot(m * geodesics_key + c, 'b-', label='geodesics')
#     ax.plot(rtts_key, 'r-', label='RTTs')
#     ax.set_title(f'{key[0]} $\\rightarrow$ {key[1]}')
#     ax.legend()
#     fig.savefig(directory_output_images / f'{key[0]}_{key[1]}.png', dpi=200)
#     fig.clear()
#     plt.close()

In [ ]:
# Write CSVs
# for key in keys:
#     rtts_key = [float(geodesics_data[key]['rtt']) if geodesics_data[key]['rtt'] else None for geodesics_data in links_data]
#     geodesics_key = [geodesics_data[key]['geodesic_distance'] for geodesics_data in links_data]

#     with open(directory_output_csvs / f'{min(key)}_{max(key)}.csv', 'w') as f:
#         writer = csv.DictWriter(f, ['time', 'rtt', 'geodesic_distance', 'geodesic_distance_transformed'])
#         writer.writeheader()

#         for t, rtt, geodesic in zip(times, rtts_key, geodesics_key):
#             writer.writerow({
#                 'time': int(t / 1000),
#                 'rtt': rtt,
#                 'geodesic_distance': geodesic,
#                 'geodesic_distance_transformed': m * geodesic + c,
#             })

In [ ]:
rtt_variations = [
    key
    for _, key in sorted([
        ((np.min(rtts_key) - np.max(rtts_key)) / (np.median(rtts_key) if np.median(rtts_key) != 0. else 1.), key)
        for key in keys
        for rtts_key in (np.array([float(geodesics_data[key]['rtt']) if geodesics_data[key]['rtt'] else 0. for geodesics_data in links_data]),)
    ])
]

In [ ]:
for key in rtt_variations[:120]:
    # print(f'{key[0]} -> {key[1]}')
    print(f'{key},')